# 1.1 Import Packages

In [1]:
import pandas as pd
import praw

# 2.1 Data Scrape

In [2]:
# Instantiate Reddit API scraper
# Taken from Ada PRAW lesson

reddit_tw = praw.Reddit(client_id = 'oyQm0DwF4Do0yQ', # Personal script key
                     client_secret = 'jYqM7fjnvbI55QRe1Xl09hb62-k', # Personal secret key
                     user_agent = 'TAW', # Personal application name
                     username = 'tawhalen', # Personal Reddit user name
                     password = '9bry2' # Personal Reddit password - Please don't post weird things
                    )

In [3]:
# Create list of target subreddits
target_subreddits = ['nfl','patriots']

# Choose target posts from list of: new, hot, top
target_posts = 'new'

In [4]:
def get_subreddit_data(subs_list, reddit_scrape, method = 'new'):
    """Returns dictionary of posts pulled from given list of subreddits
    
    Parameters
    ----------
    subs_list: list
        List of target subreddits for scrape. Must be valid subreddits.
    reddit_scrape: praw.Reddit()
        Instantiated PRAW Reddit scrape with desired parameters
    method: str
        Method to be called upon the PRAW. This determines which type of posts to scrape.
    
    Returns
    -------
    dictionary
        A dictionary including the subreddit, post title, post score, user id, url, comments number, created time (utc epoch seconds), and post body.
    
    """
    # Create dictionary of empty lists in format of reddit posts
    # Add subreddit to allow differentiation
    dict_subreddits = { "subreddit":[],
                        "title":[],
                        "score":[],
                        "id":[],
                        "url":[], 
                        "comms_num": [],
                        "created_utc": [],
                        "body":[]
                      }

    # Loop through chosen subreddits, pull data through PRAW, and add results to dictionary
    for sub in subs_list:

        # Create subreddit pull of 1000 records from chosen category
        # Dynamic method call take from:
        # https://stackoverflow.com/questions/17726180/how-to-dynamically-call-a-method-in-python
        subreddit_pull = getattr(reddit_scrape.subreddit(sub),method)(limit = 1000)

        # Create a record counter to track number of rows
        record_count = 0

        # Loop through each record and add data to the dictionary
        for item in subreddit_pull:
            dict_subreddits["subreddit"].append(sub)
            dict_subreddits["title"].append(item.title)
            dict_subreddits["score"].append(item.score)
            dict_subreddits["id"].append(item.id)
            dict_subreddits["url"].append(item.url)
            dict_subreddits["comms_num"].append(item.num_comments)
            dict_subreddits["created_utc"].append(item.created_utc)
            dict_subreddits["body"].append(item.selftext)
            record_count += 1

        # Check that at least 900 records have been pulled for each subreddit
        if record_count < 900:
            raise Exception(f'Insufficient data: the {sub} subreddit returned only {record_count} records.')
      
    # Return the completed data pull
    return dict_subreddits

In [5]:
# Convert scrape dictionaries into a dataframe
df_subreddits = pd.DataFrame(get_subreddit_data(target_subreddits, reddit_tw, target_posts))

# 3.1 Write Data

In [6]:
# Write raw subreddits data as .csv file
df_subreddits.to_csv('./data/subreddits_raw.csv',index = False)